In [ ]:
!pip install meteocalc

  Created wheel for meteocalc: filename=meteocalc-1.1.0-cp37-none-any.whl size=8195 sha256=915d39c5a7d3c9001973515105ce4c2f8aa1870fa268428ce5d9d5fec8cfdfd7
  Stored in directory: /root/.cache/pip/wheels/9e/34/13/83d36ecc28837e3c2a5b696542e697538e7c1025382f4ded55
Successfully built meteocalc


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#from google.col4/1AY0e-g4QBLBOBCwIslaAFOL4hHyTLzfMotxykEmSvl0MPtvH0qLvhmW4eIUab import drive
#drive.mount('/content/drive')
#!cp /content/drive/MyDrive/ashrae.zip ashrae.zip
#!unzip -q ashrae.zip

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.figure_factory as ff

from IPython.core.pylabtools import figsize
from plotly.offline import init_notebook_mode,iplot,plot
%matplotlib inline
init_notebook_mode(connected=True)
plt.style.use("ggplot")

from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import  LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

# importer les fichiers .py sur le notebook
%run /content/utils.py
%run /content/feature_enginering.py

In [ ]:
building = pd.read_csv('/content/drive/MyDrive/building_metadata.csv')
weather = pd.read_csv('/content/drive/MyDrive/weather_train.csv')
train = pd.read_csv('/content/drive/MyDrive/train.csv')

train.loc[(train.meter == 0),'meter_reading'] = train['meter_reading'] * 0.2931 # convertir du kwh vers kbtu

In [ ]:
building = reduce_mem_usage(building)
weather  = reduce_mem_usage(weather)
train = reduce_mem_usage(train)

In [ ]:
building = prepare_building_data(building)
weather = prepare_weather_data(weather)
train  = time_features(train)

In [ ]:
def join_data(data , weather_data , building_data):

    data = data.merge(building_data, on = 'building_id' , how='left')
    data = data.merge(weather_data , on = ['site_id','timestamp'], how= 'left')
    data = shuffle(data, random_state=0)

    return data

train = join_data(train , weather , building)

train['log_meter_reading'] = np.log(train['meter_reading'] + 1)
train['month_group'] = train['month_group'] - 1

train = train.drop(['meter_reading'] , axis= 1)

train  = reduce_mem_usage(train)
del weather, building
gc.collect()

0

In [ ]:
cols_to_drop = ['year_built' , 'floor_count' , 'timestamp']

cols_to_scale = ['square_feet' , 'air_temperature' , 'dew_temperature' ,
                 'wind_speed', 'cloud_coverage', 'sea_level_pressure', 'wind_direction',
                 'relative_humidity', 'feels_like']

cols_to_encode = ['primary_use' , 'city' , 'country']

cols_to_fill_median = ['air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
                       'sea_level_pressure','wind_direction' , 'wind_speed', 'relative_humidity' ,
                       'feels_like']

cols_to_fill_categ = ['city' , 'country' , 'is_holiday']


Filler = Data_Filler(cols_to_fill_median,
                     cols_to_fill_categ)

Filler.fit(train)
train = Filler.transform(train)

ScalerEncoder = Data_Scaler_Encoder(cols_to_scale ,
                                    cols_to_drop , 
                                    cols_to_encode)
ScalerEncoder.fit(train)
train = ScalerEncoder.transform(train)

In [ ]:
categorical_vars = ['building_id' , 'meter', 'hour', 'dayofweek' , 'month_group', 'site_id' , 
                    'primary_use', 'city', 'country', 'is_holiday']

for col in categorical_vars:
    train[col] = train[col].astype('category')

train_columns = train.columns

In [ ]:
y = train['log_meter_reading'].values

train = train.drop( ['log_meter_reading'] , axis = 1).values

train, valid, y_train, y_valid = train_test_split(train, y, test_size = 0.30, random_state=42)

del y
gc.collect()

254

In [ ]:
best_params = {}

In [ ]:
import lightgbm as lgb

In [ ]:
n_estimators = [25,50,150,500,1000]
rmses = []

for estim in n_estimators : 
    lgbr=lgb.LGBMRegressor(boosting_type='gbdt', n_estimators=estim, objective='regression')
    model=lgbr.fit(train , y_train)
    y_pred = model.predict(valid)
    rmses.append(mean_squared_error(y_valid, y_pred, squared = False))

best_params['n_estimators'] = n_estimators[rmses.index(min(rmses))]

In [ ]:
best_params

{'n_estimators': 1000}

In [ ]:
lgbr =  lgb.LGBMRegressor(boosting_type='gbdt', 
                          n_estimators=1000, 
                          objective='regression')

In [ ]:
train = np.concatenate((train , valid) , axis = 0)
y_train = np.concatenate((y_train , y_valid) , axis = 0)

In [ ]:
lgbr.fit(train , y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=1000, n_jobs=-1, num_leaves=31,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

In [ ]:
del train, y_train
gc.collect()

322

In [ ]:
train_columns

Index(['building_id', 'meter', 'hour', 'dayofweek', 'month_group', 'site_id',
       'primary_use', 'square_feet', 'air_temperature', 'cloud_coverage',
       'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
       'wind_direction', 'wind_speed', 'relative_humidity', 'feels_like',
       'city', 'country', 'is_holiday', 'log_meter_reading'],
      dtype='object')

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/test.csv')
weather_test = pd.read_csv('/content/drive/MyDrive/weather_test.csv')
building = pd.read_csv('/content/drive/MyDrive/building_metadata.csv')


weather_test  = reduce_mem_usage(weather_test)
building = reduce_mem_usage(building)
test = reduce_mem_usage(test)


weather_test = prepare_weather_data(weather_test)
building = prepare_building_data(building)
test = time_features(test)

In [ ]:
step_size = 100000
nrows = test.shape[0]
num_steps = nrows//step_size
print('nombre d\'iterations' , num_steps)

row_id = np.array([])
meter_values = np.array([])
y_pred = np.array([])

for j in range(num_steps+1):
    start_idx = j*step_size
    end_idx = (j+1)*step_size
    if j == num_steps : 
        start_idx = j*step_size
        end_idx = nrows
      
    test_tmp = test.iloc[start_idx:end_idx]
    
    test_tmp = join_data(test_tmp , weather_test , building)
    test_tmp = Filler.transform(test_tmp)
    test_tmp = ScalerEncoder.transform(test_tmp)
    test_tmp['month_group'] = test_tmp['month_group'] - 1
    
    for col in categorical_vars:
        test_tmp[col] = test_tmp[col].astype('category')

    row_id_tmp = test_tmp['row_id'].values
    row_id = np.concatenate((row_id,row_id_tmp))

    test_tmp = test_tmp.drop(['row_id'] , axis= 1)

    meter_values_tmp = test_tmp['meter'].values
    meter_values = np.concatenate((meter_values,meter_values_tmp))

    predictions = lgbr.predict(test_tmp)
    predictions = predictions.reshape(predictions.shape[0])

    y_pred = np.concatenate((y_pred,predictions))
    if j%50==0:
        print('iteration' , j)

nombre d'iterations 416
iteration 0
iteration 50
iteration 100
iteration 150
iteration 200
iteration 250
iteration 300
iteration 350
iteration 400


In [ ]:
del building, weather_test,test
gc.collect()

833

In [ ]:
pred_final = {'row_id': row_id, 'meter' : meter_values , 'meter_reading': y_pred}
pred_final = pd.DataFrame(data = pred_final)

KWH_TO_KBTU = 0.2931 
pred_final['meter_reading'] = np.exp(pred_final['meter_reading']) - 1
pred_final.loc[(pred_final.meter == 0),'meter_reading'] = pred_final['meter_reading'] / KWH_TO_KBTU # convertir du kwh vers kbtu

pred_final = pred_final.sort_values(by=['row_id'])
pred_final = pred_final.drop(['meter'] , axis = 1)
pred_final.loc[pred_final.meter_reading < 0, 'meter_reading'] = 0

In [ ]:
pred_final.row_id = pred_final.row_id.astype('int32')
pred_final

,row_id,meter_reading
52860,0,2.621977
33031,1,2.431558
83905,2,0.000000
15517,3,3.578069
46516,4,5.611755
...,...,...
41640965,41697595,38.248783
41659597,41697596,6.933911
41675115,41697597,20.599366
41629031,41697598,44.152377


In [ ]:
# pred_final.drop(['Unnamed: 0'])
pred_final

,row_id,meter_reading
52860,0,2.621977
33031,1,2.431558
83905,2,0.000000
15517,3,3.578069
46516,4,5.611755
...,...,...
41640965,41697595,38.248783
41659597,41697596,6.933911
41675115,41697597,20.599366
41629031,41697598,44.152377


In [ ]:
compression_opts = dict(method='zip',archive_name='out_lgbr.csv')  

pred_final.to_csv('out_lgbr.zip', index=False,compression=compression_opts)

In [ ]:
!cp out_lgbr.zip /content/drive/MyDrive/out_lgbr.zip